# This notebook will pretrain a BERT model on the WikiText-2 dataset

In [2]:
#let's get the data..let's try to use the Hugging Face API

from datasets import load_dataset

# Load the wikitext-2-raw-v1 configuration of the wikitext dataset
raw_datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

# The 'raw_datasets' object will now contain the dataset, typically split into
# 'train', 'validation', and 'test' splits.
# You can access them like this:
train_data = raw_datasets["train"]
validation_data = raw_datasets["validation"]
test_data = raw_datasets["test"]

print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(validation_data)}")
print(f"Number of test examples: {len(test_data)}")

README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Number of training examples: 36718
Number of validation examples: 3760
Number of test examples: 4358


In [12]:
train_data[67]

{'text': ' Perhaps because Abraham Lincoln had not yet been inaugurated as President , Captain Totten received no instructions from his superiors and was forced to withdraw his troops . He agreed to surrender the arsenal as long as the governor agreed to three provisions : \n'}

In [13]:
len(train_data)

36718

In [26]:
len(train_data[11032]["text"].split(' . '))

6

In [27]:
type(train_data)

datasets.arrow_dataset.Dataset

In [31]:
from transformers import AutoTokenizer

checkpoint="distilbert/distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer(train_data[67]["text"], truncation=True, max_length=10)

{'input_ids': [101, 3383, 2138, 8181, 5367, 2018, 2025, 2664, 2042, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [33]:
tokenizer.decode([101, 3383, 2138, 8181, 5367, 2018, 2025, 2664, 2042, 102])

'[CLS] perhaps because abraham lincoln had not yet been [SEP]'

In [34]:
tokenizer("[CLS] [SEP] [MASK]")

{'input_ids': [101, 101, 102, 103, 102], 'attention_mask': [1, 1, 1, 1, 1]}

In [35]:
tokenizer.decode([101, 101, 102, 103, 102])

'[CLS] [CLS] [SEP] [MASK] [SEP]'

In [37]:
tokenizer("[UNK]")

{'input_ids': [101, 100, 102], 'attention_mask': [1, 1, 1]}

In [38]:
from transformers import AutoTokenizer

checkpoint  = "google-bert/bert-base-cased"

tokenizer_bert = AutoTokenizer.from_pretrained(checkpoint)

In [45]:
tokenizer_bert(["[CLS] [SEP] [MASK]."] ,["[UNK]"])

{'input_ids': [[101, 101, 102, 103, 119, 102, 100, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1]]}